In [1]:
import sys
sys.executable

'/Users/edwardterrell/opt/anaconda3/envs/c_scrape2/bin/python'

In [2]:
from shapely.geometry import Point, Polygon
import pandas as pd
import geopandas as gpd
from zipfile import ZipFile
from datetime import datetime as dt
today = dt.today().strftime("%d-%b-%y")

from pathlib import Path
from tools import tree


In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
dt.today()
today

datetime.datetime(2020, 7, 4, 16, 14, 48, 950395)

'04-Jul-20'

In [5]:
RAW_DATA = Path("../data/raw/")
INTERIM_DATA = Path("../data/intermin/")
PROCESSED_DATA = Path("../data/processed/")
FINAL_DATA = Path("../data/final/")
EXTERNAL_DATA = Path("../data/external/")

In [6]:
tree(EXTERNAL_DATA)

+ ../data/external
    + 4-8-2020---748.zip
    + processed
            + acs5_2018_race_counties-checkpoint.csv
        + acs5_2018_race_counties.csv
    + raw
            + acs5_2018_race_counties-checkpoint.csv
        + acs5_2018_race_counties.csv
    + tl_2019_us_county.zip


In [8]:
df = pd.read_csv(EXTERNAL_DATA/'processed'/'acs5_2018_race_counties.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 44 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   geoid                                                3220 non-null   int64  
 1   name                                                 3220 non-null   object 
 2   universe                                             3220 non-null   float64
 3   universe_annotation                                  0 non-null      float64
 4   universe_moe                                         3220 non-null   float64
 5   universe_moe_annotation                              3104 non-null   object 
 6   white_alone                                          3220 non-null   float64
 7   white_alone_annotation                               0 non-null      float64
 8   white_alone_moe                                      3220 non-null  

In [10]:
df.head().T

,0,1,2,3,4
geoid,1001,1003,1005,1007,1009
name,"Autauga County, Alabama","Baldwin County, Alabama","Barbour County, Alabama","Bibb County, Alabama","Blount County, Alabama"
universe,55200,208107,25782,22527,57645
universe_annotation,NaN,NaN,NaN,NaN,NaN
universe_moe,-5.55556e+08,-5.55556e+08,-5.55556e+08,-5.55556e+08,-5.55556e+08
universe_moe_annotation,*****,*****,*****,*****,*****
white_alone,41412,172768,11898,16801,50232
white_alone_annotation,NaN,NaN,NaN,NaN,NaN
white_alone_moe,59,227,22,22,229
white_alone_moe_annotation,NaN,NaN,NaN,NaN,NaN


In [11]:
df.columns

Index(['geoid', 'name', 'universe', 'universe_annotation', 'universe_moe',
       'universe_moe_annotation', 'white_alone', 'white_alone_annotation',
       'white_alone_moe', 'white_alone_moe_annotation', 'black_alone',
       'black_alone_annotation', 'black_alone_moe',
       'black_alone_moe_annotation', 'american_indian_and_alaska_native',
       'american_indian_and_alaska_native_annotation',
       'american_indian_and_alaska_native_moe',
       'american_indian_and_alaska_native_moe_annotation', 'asian_alone',
       'asian_alone_annotation', 'asian_alone_moe',
       'asian_alone_moe_annotation', 'native_hawaiian_and_pacific_islander',
       'native_hawaiian_and_pacific_islander_annotation',
       'native_hawaiian_and_pacific_islander_moe',
       'native_hawaiian_and_pacific_islander_moe_annotation', 'other_alone',
       'other_alone_annotation', 'other_alone_moe',
       'other_alone_moe_annotation', 'two_or_more_races',
       'two_or_more_races_annotation', 'two_or_more

In [22]:
cols = [col for col in df.columns if not col.endswith('moe') | col.endswith('annotation')]

In [23]:
working_df = df[cols].copy()
working_df

,geoid,name,universe,white_alone,black_alone,american_indian_and_alaska_native,asian_alone,native_hawaiian_and_pacific_islander,other_alone,two_or_more_races,latino_alone,state,county,asians_all,other_all
0,1001,"Autauga County, Alabama",55200.0,41412.0,10475.0,159.0,568.0,5.0,41.0,1012.0,1528.0,1,1,573.0,1212.0
1,1003,"Baldwin County, Alabama",208107.0,172768.0,19529.0,1398.0,1668.0,9.0,410.0,2972.0,9353.0,1,3,1677.0,4780.0
2,1005,"Barbour County, Alabama",25782.0,11898.0,12199.0,63.0,85.0,1.0,86.0,344.0,1106.0,1,5,86.0,493.0
3,1007,"Bibb County, Alabama",22527.0,16801.0,4974.0,8.0,37.0,0.0,0.0,160.0,547.0,1,7,37.0,168.0
4,1009,"Blount County, Alabama",57645.0,50232.0,820.0,124.0,198.0,18.0,174.0,818.0,5261.0,1,9,216.0,1116.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,48283,"La Salle County, Texas",7409.0,1125.0,94.0,0.0,12.0,0.0,0.0,74.0,6104.0,48,283,12.0,74.0
3216,31113,"Logan County, Nebraska",886.0,855.0,0.0,2.0,0.0,0.0,0.0,3.0,26.0,31,113,0.0,5.0
3217,31171,"Thomas County, Nebraska",645.0,629.0,0.0,3.0,0.0,0.0,0.0,10.0,3.0,31,171,0.0,13.0
3218,30033,"Garfield County, Montana",1141.0,1139.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,30,33,0.0,0.0


# Checkpoint

In [24]:
working_df.to_csv(PROCESSED_DATA/'counties.csv', index=False, encoding='utf-8' )